In [ ]:
"""
Abdelali SAADALI II-BDCC
Asaad FETHALLAH II-CCN
Marouane MOUNIR II-BDCC
Muhammed Irfan WAHYUDI II-BDCC
"""
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

# -------------------------
# 1. Charger plusieurs CSV
# -------------------------
files = [
    "aggregated2/67_with_future_months.csv",
    "aggregated2/68_with_future_months.csv",
    "aggregated2/73_with_future_months.csv",
    "aggregated2/74_with_future_months.csv",
    "aggregated2/75_with_future_months.csv",
    "aggregated2/76_with_future_months.csv",
    "aggregated2/78_with_future_months.csv",
    "aggregated2/79_with_future_months.csv",
    "aggregated2/90_with_future_months.csv",
    "aggregated2/91_with_future_months.csv",
    "aggregated2/114_with_future_months.csv",
    "aggregated2/116_with_future_months.csv",
    "aggregated2/119_with_future_months.csv",
    "aggregated2/121_with_future_months.csv",
    "aggregated2/130_with_future_months.csv",
    "aggregated2/132_with_future_months.csv",
    "aggregated2/150_with_future_months.csv",
    "aggregated2/151_with_future_months.csv",
    "aggregated2/229_with_future_months.csv",
    "aggregated2/280_with_future_months.csv"
]

dfs = [pd.read_csv(f) for f in files]
df = pd.concat(dfs, ignore_index=True)

# -------------------------
# 2. Convertir month → numéro
# -------------------------
month_map = {
    'janvier':1,'février':2,'mars':3,'avril':4,'mai':5,'juin':6,
    'juillet':7,'août':8,'septembre':9,'octobre':10,'novembre':11,'décembre':12
}

df['month_number'] = df['month'].apply(lambda x: month_map[x.split()[0]])
df['year'] = df['month'].apply(lambda x: int(x.split()[1]))

# -------------------------
# 3. Fonction d'entraînement
# -------------------------
def train_single_model(X, y, lr=1e-2, max_epochs=2000, tol=1e-6):
    
    if np.std(y) == 0:  # salaire constant
        return {
            "w": np.zeros((X.shape[1]+1,1)),
            "X_mean": X.mean(axis=0),
            "X_std": X.std(axis=0),
            "y_mean": y[0,0],
            "y_std": 1.0,
            "mse": 0.0
        }

    # Ajouter biais
    X_bias = np.hstack([np.ones((X.shape[0],1)), X])

    # Normalisation
    X_mean = X_bias[:,1:].mean(axis=0)
    X_std  = X_bias[:,1:].std(axis=0)
    X_norm = X_bias.copy()
    X_norm[:,1:] = (X_bias[:,1:] - X_mean) / X_std

    y_mean = y.mean()
    y_std = y.std()
    y_norm = (y - y_mean) / y_std

    w = np.zeros((X_norm.shape[1],1))
    prev_loss = np.inf
    final_loss = None

    for _ in range(max_epochs):
        y_pred = X_norm @ w
        error = y_pred - y_norm
        grad = (2/len(X_norm)) * (X_norm.T @ error)
        w -= lr * grad

        loss = np.mean(error**2)
        final_loss = loss
        if abs(prev_loss - loss) < tol:
            break
        prev_loss = loss

    return {
        "w": w,
        "X_mean": X_mean,
        "X_std": X_std,
        "y_mean": y_mean,
        "y_std": y_std,
        "mse": float(final_loss)
    }

# -------------------------
# 4. Entraîner un modèle par salarié (progression générale)
# -------------------------
models = {}
names = df["full_name"].unique()

print("🔧 Entraînement d'un modèle par salarié...\n")

for name in tqdm(names, desc="Employees", ncols=70):
    sub = df[df["full_name"] == name]
    X = sub[["month_number", "year"]].values
    y = sub["salaire"].values.reshape(-1,1)

    models[name] = train_single_model(X, y)

# -------------------------
# 5. Sauvegarde
# -------------------------
with open("Asaad_Salaries_Models.pkl", "wb") as f:
    pickle.dump({"models": models, "month_map": month_map}, f)

print("\n✅ Tous les modèles ont été sauvegardés dans Asaad_Salaries_Models.pkl")

# -------------------------
# 6. Fonction de prédiction
# -------------------------
def predict(full_name, month, year):
    if full_name not in models:
        print("❌ Salarié inconnu.")
        return None

    m = models[full_name]
    x = np.array([month_map[month], year])
    x = (x - m["X_mean"]) / m["X_std"]
    x = np.hstack([1, x]).reshape(1,-1)

    y_norm = x @ m["w"]
    y = float(y_norm * m["y_std"] + m["y_mean"])
    return y

# -------------------------
# 7. Exemple
# -------------------------
print("\nExemple de prédiction :")
print("Salaire prédit pour Haidari Hassan en mars 2025 :",
      predict("Haidari Hassan", "mars", 2025))


🔧 Entraînement d'un modèle par salarié...



Employees: 100%|████████████████| 31845/31845 [11:03<00:00, 48.01it/s]



✅ Tous les modèles ont été sauvegardés dans Asaad_Salaries_Models.pkl

Exemple de prédiction :
Salaire prédit pour Haidari Hassan en mars 2025 : 18136.984430654687


C:\Users\hp\AppData\Local\Temp\ipykernel_11716\3891645776.py:136: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y = float(y_norm * m["y_std"] + m["y_mean"])


In [12]:
print("Salaire prédit pour Lazraq Zouhair en mars 2025 :",
      predict("Lazraq Zouhair", "octobre", 2025))

Salaire prédit pour Lazraq Zouhair en mars 2025 : 29853.528403737226


C:\Users\hp\AppData\Local\Temp\ipykernel_11716\3891645776.py:136: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y = float(y_norm * m["y_std"] + m["y_mean"])
